# NLP - Part 2 - Chatbot

In [1]:
# Importing dependencies

import numpy as np
from sklearn.preprocessing import LabelEncoder

import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 
import re

import keras, tensorflow
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalAveragePooling1D

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import BatchNormalization, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences

import json
import random
import pickle

from nltk.stem import WordNetLemmatizer 
import string

## Import data

In [2]:
# import corpus file
with open('GL Bot.json') as file:
    corpus = json.load(file)

# Display corpus file
print(corpus)

{'intents': [{'tag': 'Intro', 'patterns': ['hi', 'how are you', 'is anyone there', 'hello', 'whats up', 'hey', 'yo', 'listen', 'please help me', 'i am learner from', 'i belong to', 'aiml batch', 'aifl batch', 'i am from', 'my pm is', 'blended', 'online', 'i am from', 'hey ya', 'talking to you for first time'], 'responses': ['Hello! how can i help you ?'], 'context_set': ''}, {'tag': 'Exit', 'patterns': ['thank you', 'thanks', 'cya', 'see you', 'later', 'see you later', 'goodbye', 'i am leaving', 'have a Good day', 'you helped me', 'thanks a lot', 'thanks a ton', 'you are the best', 'great help', 'too good', 'you are a good learning buddy'], 'responses': ['I hope I was able to assist you, Good Bye'], 'context_set': ''}, {'tag': 'Olympus', 'patterns': ['olympus', 'explain me how olympus works', 'I am not able to understand olympus', 'olympus window not working', 'no access to olympus', 'unable to see link in olympus', 'no link visible on olympus', 'whom to contact for olympus', 'lot of p

In [3]:
# List initialization
corp1 = corpus["intents"]
corp2 = [
    {'tag': 'options', 
     'patterns': ["How can you help me", "What can you do", "What help you provide", "What do you know"],
     'responses': ["I can guide you with machine learning and artificial learning concepts", "I can assist you with your program"], 
     'context_set': ''
    },
    {'tag': 'SL', 
     'patterns': ['bias-variance tradeoff', 'support vector machines', 'normalization', 'Can you help me understand svm', 
                  'could you please explain me how machine learning works', 'K nearest neighbor algorithm'
                  'Could me help me understand naive bayes', 'decision trees', 'labeled data', 'supervised', 'Do you know KNN',
                  'Can you help me with KNN algorithm', 'Can you explain KNN', 'Can you help me with supervised ML?'], 
     'responses': ['Link: https://en.wikipedia.org/wiki/Supervised_learning'], 
     'context_set': ''
    },
    {'tag': 'USL', 
     'patterns': ['PCA', 'Pricipal component analysis', 'K means clustering', 'dimensionality reduction', 'unlabeled data'
                  'clustering', 'K nearest neighbor algorithm', 'clustering', 'association', 'hierarchical clustering'
                  'Singular value decomposition', 'Do you know PCA'], 
     'responses': ['Link: https://en.wikipedia.org/wiki/Unsupervised_learning'], 
     'context_set': ''
    },
    {'tag': 'NN', 
     'patterns': ['RelU', 'sigmoid function', 'tanh', 'optimizer', 'neural networks', 'Can you help me with deep learning',
                  'ANN', 'CNN', 'Transfer learning'],
     'responses': ['Link: https://en.wikipedia.org/wiki/Artificial_neural_network'],
     'context_set': ''
    },
    {'tag': 'Bot', 
     'patterns': ['May I know who you are', "whats you're name"],
     'responses': ["I'm Proton, your virtual assistant and I'm 24x7 available"],
     'context_set': ''
    },
    {'tag': 'no answer', 
     'patterns': [],
     'responses': ["Sorry, I can't understand you", "Please give me more info", "Unable to understand you, Can you provide info"], 
     'context_set': ''
    },
    {'tag': 'Intro', 
     'patterns': ['How do you do', 'Hi.. Can you help me with my issue', 'Are you there', "what's up", "Howdy", 
                  'Could you please help me', 'I need you to listen to me', 'I belong to a new batch of', 
                  'Artificial Intelligence class', 'AI & ML batch', 'Machine Learning batch', 'My Program Manager is'], 
     'responses': ['Hi there, how can I help you', 'Good to see you again', 'Hello thanks for asking', 
                   'Hello, How may I help you'], 
     'context_set': ''
    },
    {'tag': 'Exit',
     'patterns': ["that was very helpful", 'thanks again', 'awesome, thanks', 'thanks for helping me out', 
                  'will catch you later', "i'll take a leave", 'have a Great day', 'you helped a lot', 
                  'you are a great learning buddy', 'bye', 'good bye'],
     'responses': ['Anytime!!', "Happy to help!", "My pleasure!", "Glad I could assist you"],
     'context_set': ''
    },
    {'tag': 'Ticket', 
     'patterns': ['my issue is not solved', 'you answer did not help', 'I did not get the answer', 'solution was not good', 
                  'you did not help', 'I want to talk to a my PM', 'connect me to the program manager', 
                  'I want to raise a ticket', 'no'],
     'responses': ['Transferring the request to your PM', 'Please wait while I transfer your request to PM'], 
     'context_set': ''
    }
    ]

In [4]:
# Iterating and using extend to convert
for elm2 in corp2:
  
    for elm1 in corp1:
        if elm2['tag'] == elm1['tag']:
            elm1['responses'].extend(elm2['responses'])
            break
    else:
        corp1.append(elm2)
        
# printing 
# print(corp1)

In [5]:
corpus["intents"] = corp1

In [6]:
print(corpus['intents'])

[{'tag': 'Intro', 'patterns': ['hi', 'how are you', 'is anyone there', 'hello', 'whats up', 'hey', 'yo', 'listen', 'please help me', 'i am learner from', 'i belong to', 'aiml batch', 'aifl batch', 'i am from', 'my pm is', 'blended', 'online', 'i am from', 'hey ya', 'talking to you for first time'], 'responses': ['Hello! how can i help you ?', 'Hi there, how can I help you', 'Good to see you again', 'Hello thanks for asking', 'Hello, How may I help you'], 'context_set': ''}, {'tag': 'Exit', 'patterns': ['thank you', 'thanks', 'cya', 'see you', 'later', 'see you later', 'goodbye', 'i am leaving', 'have a Good day', 'you helped me', 'thanks a lot', 'thanks a ton', 'you are the best', 'great help', 'too good', 'you are a good learning buddy'], 'responses': ['I hope I was able to assist you, Good Bye', 'Anytime!!', 'Happy to help!', 'My pleasure!', 'Glad I could assist you'], 'context_set': ''}, {'tag': 'Olympus', 'patterns': ['olympus', 'explain me how olympus works', 'I am not able to und

## Data pre-processing

In [7]:
# Extract required data from the corpus
train_sentences = [] #tokens
train_labels = []    #identified tags or labels

labels = []          
responses = []       

for intent in corpus ['intents']:
    for pattern in intent['patterns']:
        
        train_sentences.append(pattern)
        train_labels.append(intent['tag'])
        
    responses.append(intent['responses'])
        
    #Add missing tag if any
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [8]:
# print(train_sentences) #patterns
# print(train_labels) #all tags
# print(labels) #single tags
# print(responses) #responses

We use “LabelEncoder()” function provided by scikit-learn to convert the target labels into a model understandable form.

In [9]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(train_labels)
train_labels = lbl_encoder.transform(train_labels)

Let us vectorize our text data corpus by using the “Tokenizer” class and it allows us to limit our vocabulary size up to some defined number. When we use this class for the text pre-processing task, by default all punctuations will be removed, turning the texts into space-separated sequences of words, and these sequences are then split into lists of tokens. They will then be indexed or vectorized. We can also add “oov_token” which is a value for “out of token” to deal with out of vocabulary words(tokens) at inference time. The “pad_sequences” method is used to make all the training text sequences into the same size.

In [10]:
vocab_size = 100000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(train_sentences)

word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(train_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

## Model training

In [11]:
model = Sequential()

model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 16)            1600000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 11)                187       
Total params: 1,600,731
Trainable params: 1,600,731
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Fit  the model
epochs = 500
history = model.fit(padded_sequences, np.array(train_labels), epochs=epochs)

Epoch 1/500
5/5 [==============================] - 0s 16ms/step - loss: 2.3957 - accuracy: 0.0846
Epoch 2/500
5/5 [==============================] - 0s 15ms/step - loss: 2.3905 - accuracy: 0.2084
Epoch 3/500
5/5 [==============================] - 0s 16ms/step - loss: 2.3846 - accuracy: 0.1822
Epoch 4/500
5/5 [==============================] - 0s 17ms/step - loss: 2.3774 - accuracy: 0.2165
Epoch 5/500
5/5 [==============================] - 0s 19ms/step - loss: 2.3710 - accuracy: 0.2126
Epoch 6/500
5/5 [==============================] - 0s 16ms/step - loss: 2.3648 - accuracy: 0.1853
Epoch 7/500
5/5 [==============================] - 0s 15ms/step - loss: 2.3601 - accuracy: 0.1701
Epoch 8/500
5/5 [==============================] - 0s 16ms/step - loss: 2.3436 - accuracy: 0.2065
Epoch 9/500
5/5 [==============================] - 0s 16ms/step - loss: 2.3475 - accuracy: 0.1961
Epoch 10/500
5/5 [==============================] - 0s 16ms/step - loss: 2.3332 - accuracy: 0.1848
Epoch 11/500
5/5 [=

5/5 [==============================] - 0s 14ms/step - loss: 1.5917 - accuracy: 0.3893
Epoch 84/500
5/5 [==============================] - 0s 14ms/step - loss: 1.5431 - accuracy: 0.3759
Epoch 85/500
5/5 [==============================] - 0s 14ms/step - loss: 1.6102 - accuracy: 0.3641
Epoch 86/500
5/5 [==============================] - 0s 15ms/step - loss: 1.5962 - accuracy: 0.4033
Epoch 87/500
5/5 [==============================] - 0s 15ms/step - loss: 1.5950 - accuracy: 0.3803
Epoch 88/500
5/5 [==============================] - 0s 14ms/step - loss: 1.5368 - accuracy: 0.4461
Epoch 89/500
5/5 [==============================] - 0s 15ms/step - loss: 1.5363 - accuracy: 0.3774
Epoch 90/500
5/5 [==============================] - 0s 15ms/step - loss: 1.4713 - accuracy: 0.4132
Epoch 91/500
5/5 [==============================] - 0s 15ms/step - loss: 1.5116 - accuracy: 0.4325
Epoch 92/500
5/5 [==============================] - 0s 15ms/step - loss: 1.5446 - accuracy: 0.4463
Epoch 93/500
5/5 [=====

5/5 [==============================] - 0s 16ms/step - loss: 0.5742 - accuracy: 0.8787
Epoch 166/500
5/5 [==============================] - 0s 17ms/step - loss: 0.5393 - accuracy: 0.8884
Epoch 167/500
5/5 [==============================] - 0s 17ms/step - loss: 0.5089 - accuracy: 0.9203
Epoch 168/500
5/5 [==============================] - 0s 16ms/step - loss: 0.5404 - accuracy: 0.8778
Epoch 169/500
5/5 [==============================] - 0s 16ms/step - loss: 0.4937 - accuracy: 0.9051
Epoch 170/500
5/5 [==============================] - 0s 15ms/step - loss: 0.5538 - accuracy: 0.8697
Epoch 171/500
5/5 [==============================] - 0s 15ms/step - loss: 0.4962 - accuracy: 0.9019
Epoch 172/500
5/5 [==============================] - 0s 16ms/step - loss: 0.4887 - accuracy: 0.8858
Epoch 173/500
5/5 [==============================] - 0s 15ms/step - loss: 0.5185 - accuracy: 0.8882
Epoch 174/500
5/5 [==============================] - 0s 15ms/step - loss: 0.4555 - accuracy: 0.9118
Epoch 175/500


5/5 [==============================] - 0s 18ms/step - loss: 0.1666 - accuracy: 0.9927
Epoch 248/500
5/5 [==============================] - 0s 15ms/step - loss: 0.1799 - accuracy: 0.9866
Epoch 249/500
5/5 [==============================] - 0s 14ms/step - loss: 0.1502 - accuracy: 0.9977
Epoch 250/500
5/5 [==============================] - 0s 16ms/step - loss: 0.1435 - accuracy: 0.9964
Epoch 251/500
5/5 [==============================] - 0s 14ms/step - loss: 0.1686 - accuracy: 0.9868
Epoch 252/500
5/5 [==============================] - 0s 15ms/step - loss: 0.1470 - accuracy: 0.9920
Epoch 253/500
5/5 [==============================] - 0s 16ms/step - loss: 0.1442 - accuracy: 0.9940
Epoch 254/500
5/5 [==============================] - 0s 15ms/step - loss: 0.1378 - accuracy: 0.9814
Epoch 255/500
5/5 [==============================] - 0s 15ms/step - loss: 0.1334 - accuracy: 0.9868
Epoch 256/500
5/5 [==============================] - 0s 17ms/step - loss: 0.1293 - accuracy: 0.9964
Epoch 257/500


5/5 [==============================] - 0s 16ms/step - loss: 0.0574 - accuracy: 1.0000
Epoch 330/500
5/5 [==============================] - 0s 15ms/step - loss: 0.0495 - accuracy: 1.0000
Epoch 331/500
5/5 [==============================] - 0s 15ms/step - loss: 0.0514 - accuracy: 1.0000
Epoch 332/500
5/5 [==============================] - 0s 15ms/step - loss: 0.0443 - accuracy: 1.0000
Epoch 333/500
5/5 [==============================] - 0s 14ms/step - loss: 0.0503 - accuracy: 1.0000
Epoch 334/500
5/5 [==============================] - 0s 15ms/step - loss: 0.0502 - accuracy: 1.0000
Epoch 335/500
5/5 [==============================] - 0s 17ms/step - loss: 0.0488 - accuracy: 1.0000
Epoch 336/500
5/5 [==============================] - 0s 17ms/step - loss: 0.0431 - accuracy: 1.0000
Epoch 337/500
5/5 [==============================] - 0s 16ms/step - loss: 0.0510 - accuracy: 1.0000
Epoch 338/500
5/5 [==============================] - 0s 16ms/step - loss: 0.0455 - accuracy: 1.0000
Epoch 339/500


5/5 [==============================] - 0s 17ms/step - loss: 0.0216 - accuracy: 1.0000
Epoch 412/500
5/5 [==============================] - 0s 19ms/step - loss: 0.0227 - accuracy: 1.0000
Epoch 413/500
5/5 [==============================] - 0s 16ms/step - loss: 0.0215 - accuracy: 1.0000
Epoch 414/500
5/5 [==============================] - 0s 20ms/step - loss: 0.0208 - accuracy: 1.0000
Epoch 415/500
5/5 [==============================] - 0s 22ms/step - loss: 0.0209 - accuracy: 1.0000
Epoch 416/500
5/5 [==============================] - 0s 17ms/step - loss: 0.0212 - accuracy: 1.0000
Epoch 417/500
5/5 [==============================] - 0s 17ms/step - loss: 0.0210 - accuracy: 1.0000
Epoch 418/500
5/5 [==============================] - 0s 17ms/step - loss: 0.0178 - accuracy: 1.0000
Epoch 419/500
5/5 [==============================] - 0s 16ms/step - loss: 0.0183 - accuracy: 1.0000
Epoch 420/500
5/5 [==============================] - 0s 19ms/step - loss: 0.0178 - accuracy: 1.0000
Epoch 421/500


5/5 [==============================] - 0s 17ms/step - loss: 0.0085 - accuracy: 1.0000
Epoch 494/500
5/5 [==============================] - 0s 17ms/step - loss: 0.0091 - accuracy: 1.0000
Epoch 495/500
5/5 [==============================] - 0s 16ms/step - loss: 0.0093 - accuracy: 1.0000
Epoch 496/500
5/5 [==============================] - 0s 16ms/step - loss: 0.0085 - accuracy: 1.0000
Epoch 497/500
5/5 [==============================] - 0s 16ms/step - loss: 0.0099 - accuracy: 1.0000
Epoch 498/500
5/5 [==============================] - 0s 16ms/step - loss: 0.0081 - accuracy: 1.0000
Epoch 499/500
5/5 [==============================] - 0s 16ms/step - loss: 0.0083 - accuracy: 1.0000
Epoch 500/500
5/5 [==============================] - 0s 16ms/step - loss: 0.0085 - accuracy: 1.0000


In [13]:
# to save the trained model
model.save("Proton_bot")

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: Proton_bot\assets


## Inference

The chatbot will calculate the similarity between the new text sequence and training data. Considering the confidence scores got for each category, it categorizes the user message to an intent with the highest confidence score.

In [14]:
def chat():
    # load trained model
    model = keras.models.load_model('Proton_bot')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)
    
    # parameters
    max_len = 20
    
    print ("Welcome to Great Learning.")
    print ("I'm Proton, your GL virtual assitant. How may I help you?")
    print ("Note: You can type 'quit' to end the session")
    
    # While loop to run the chatbot indefinetely
    while True:
        print("")
        print("User: ", end="")
        
        # Takes the user input and converts all characters to lowercase
        inp = input()
         
        # Defining the Chatbot's exit condition
        if inp.lower() == "quit":
            print("Proton: Thank you for visiting Great Learning support. Stay safe.")
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])


        count = 0
        matched_intent = None
        
        for i in corpus['intents']:
             
            if i['tag'] == tag:
                print("Proton:", np.random.choice(i['responses']))

chat()

Welcome to Great Learning.
I'm Proton, your GL virtual assitant. How may I help you?
Note: You can type 'quit' to end the session

User: Hi
Proton: Hello! how can i help you ?

User: Who are you
Proton: I'm Proton, your virtual assistant and I'm 24x7 available

User: What can you do?
Proton: I can assist you with your program

User: Could me help me understand naive bayes?
Proton: Link: Machine Learning wiki 

User: Great thank you
Proton: My pleasure!

User: quit
Proton: Thank you for visiting Great Learning support. Stay safe.
